In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.tsa.arima.model import ARIMA
from utils import *

In [11]:
filtrar_ub()

NameError: name 'filtrar_ub' is not defined

In [3]:
import importlib
import utils
importlib.reload(utils)

from utils import filtrar_ub


ImportError: cannot import name 'filtrar_ub' from 'utils' (w:\repo\Price_prediction\utils.py)

# Importar datos

In [4]:
df = pd.read_csv("bigqueryDatabases/prices.csv")
df.head()

,date,price,priceId,priceName,currency,currencyId,priceType,source
0,2025-05-27,12.676565,14,UB Canada WC Whole FOB Miami,USD,0,EXPORTS,URNER BARRY
1,2022-10-01,14.637188,4,Japan Frozen Atlantic TRIM-E,USD,0,EXPORT,CHILEAN EXPORTS
2,2023-03-01,12.265160,5,Japan Fresh Atlantic TRIM-A,USD,0,EXPORT,CHILEAN EXPORTS
3,2024-02-01,5.859417,1,Japan Frozen Coho HG,USD,0,EXPORT,CHILEAN EXPORTS
4,2012-11-08,6.724091,11,UB Atlantic TRIM-D 3-4 Lb FOB Miami,USD,0,EXPORTS,URNER BARRY


# Limpieza

In [5]:
"""
df["date"] = pd.to_datetime(df["date"])

# Urner Barry 
producto = "UB Atlantic TRIM-D 3-4 Lb FOB Miami"
df = df[df["priceName"] == producto].copy()

# YYYY-MM
df["year_month"] = df["date"].dt.to_period("M").astype(str)

# Agrupar por mes 
serie_mensual = df.groupby("year_month")["price"].mean().reset_index()
"""

'\ndf["date"] = pd.to_datetime(df["date"])\n\n# Urner Barry \nproducto = "UB Atlantic TRIM-D 3-4 Lb FOB Miami"\ndf = df[df["priceName"] == producto].copy()\n\n# YYYY-MM\ndf["year_month"] = df["date"].dt.to_period("M").astype(str)\n\n# Agrupar por mes \nserie_mensual = df.groupby("year_month")["price"].mean().reset_index()\n'

In [4]:
from utils import *

In [5]:
!pwd

"pwd" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


In [6]:

serie_mensual = filtrar_ub(df)
serie_mensual.head()


NameError: name 'filtrar_ub' is not defined

In [7]:
df

,year_month,price
0,2007-01,9.198165
1,2007-02,9.286962
2,2007-03,9.051190
3,2007-04,9.273183
4,2007-05,9.060988
...,...,...
216,2025-01,14.121816
217,2025-02,13.820212
218,2025-03,13.902885
219,2025-04,14.183055
